In [1]:
import os 
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.stem.porter import *

from sklearn.feature_extraction.text import CountVectorizer


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [2]:
data=pd.read_csv('dataset.csv',encoding = "ISO-8859-1")

In [3]:
data.head()

,SentimentText,Sentiment
0,"first think another Disney movie, might good, ...",1
1,"Put aside Dr. House repeat missed, Desperate H...",0
2,"big fan Stephen King's work, film made even gr...",1
3,watched horrid thing TV. Needless say one movi...,0
4,truly enjoyed film. acting terrific plot. Jeff...,1


In [4]:
def remove_pattern(input_text,pattern):
  r=re.findall(pattern,input_text)
  for i in r:
    input_text=re.sub(i,'',input_text)
    
  return input_text

In [5]:
data['tidy']=np.vectorize(remove_pattern)(data['SentimentText'],',[\w]*')
data['tidy']=data['tidy'].str.replace('[^a-zA-z]'," ")
data['tidy']=data['tidy'].apply(lambda x:' '.join([w for w in x.split() if len(w)>2]))

In [6]:
data.head()

,SentimentText,Sentiment,tidy
0,"first think another Disney movie, might good, ...",1,first think another Disney movie might good ki...
1,"Put aside Dr. House repeat missed, Desperate H...",0,Put aside House repeat missed Desperate Housew...
2,"big fan Stephen King's work, film made even gr...",1,big fan Stephen King work film made even great...
3,watched horrid thing TV. Needless say one movi...,0,watched horrid thing Needless say one movies w...
4,truly enjoyed film. acting terrific plot. Jeff...,1,truly enjoyed film acting terrific plot Jeff C...


In [7]:
tokenized_review=data['tidy'].apply(lambda x:x.split())
tokenized_review.head()

0    [first, think, another, Disney, movie, might, ...
1    [Put, aside, House, repeat, missed, Desperate,...
2    [big, fan, Stephen, King, work, film, made, ev...
3    [watched, horrid, thing, Needless, say, one, m...
4    [truly, enjoyed, film, acting, terrific, plot,...
Name: tidy, dtype: object

In [8]:
stemmer=PorterStemmer()

In [ ]:
tokenized_review=tokenized_review.apply(lambda x:[stemmer.stem(i) for i in x])

In [ ]:
tokenized_review.head()

In [ ]:
for i in range(len(tokenized_review)):
    tokenized_review[i]=' '.join(tokenized_review[i])

data['tidy']=tokenized_review

In [ ]:
data.head()

In [ ]:
all_words=' '.join([word for word in data['tidy']])

In [ ]:
all_words_list=all_words.split(' ')

In [ ]:
all_word_frame=pd.DataFrame(all_words_list)

In [ ]:
X=all_word_frame[0].value_counts()

In [ ]:
X

In [ ]:
df2 = pd.DataFrame(np.array(all_words.split(' ')).reshape(-1, 1), columns=['words'])

In [ ]:
unique_words = list(df2['words'].str.split(' ', expand=True).stack().unique())

In [ ]:
len(unique_words)

In [ ]:
word_counts = list(df2['words'].value_counts())

In [ ]:
df4 = pd.DataFrame(np.hstack((np.array(unique_words).reshape(-1,1),np.array(word_counts).reshape(-1,1))), columns=['words','word_count'])

In [ ]:
df4_new=df4[df4['word_count'].map(len)==1]

In [ ]:
df4_words_list = list(df4_new['words'])

In [ ]:
unique_words=list(unique_words)

In [ ]:
for word in df4_words_list:
    unique_words.remove(word)

In [ ]:
len(unique_words)

In [ ]:
type(unique_words)

In [ ]:
df4_new.head()

In [ ]:
new_data=data

In [ ]:
new_data.head()

In [ ]:
tokenized_tidy=new_data['tidy'].apply(lambda x:x.split())
tokenized_tidy.head()

In [ ]:
for i in tokenized_tidy:
    for j in i:
        if j not in unique_words:
            i.remove(j)

In [ ]:
len(tokenized_tidy)

In [ ]:
for i in range(len(tokenized_tidy)):
    tokenized_tidy[i]=' '.join(tokenized_tidy[i])

new_data['new_tidy']=tokenized_tidy

In [ ]:
new_data.head()

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
rev_vectorizer=CountVectorizer(stop_words='english')
rev= rev_vectorizer.fit_transform(new_data['new_tidy'])

In [ ]:
train_rev=rev

xtrain_rev,xvalid_rev,ytrain,yvalid=train_test_split(train_rev,new_data['Sentiment'],test_size=0.2)

In [ ]:
lreg=LogisticRegression()
lreg.fit(xtrain_rev,ytrain)

prediction = lreg.predict_proba(xvalid_rev) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.5 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)

In [ ]:
f1_score(yvalid, prediction_int)

In [ ]:
lreg.fit(xtrain_rev,ytrain)

In [ ]:
y_pred=lreg.predict(xvalid_rev)

In [ ]:
print(accuracy_score(yvalid,y_pred))